PASO 1: Cargar el archivo Excel (ETL: Extract)

In [2]:
# Cargar librerias para comenzar con la lectura del data set
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

# Cargar el archivo 
df = pd.read_excel("sismos_completo.xlsx")

# Primer vistazo
df.shape
df.head()


,#,Año,M,D,H,M.1,S,Err.,L,Latitud,Err..1,Longitud,Err..2,Prof.,Err..3,NST,RMS,GAP,ML,Agen.
0,1,2021,1,3,23,35,33.8,0.30,L,-54.804,2.3,-67.206,2.5,0.0,0.0,1,0.0,360,1.6,EAR
1,2,2021,1,26,7,46,42.3,3.16,R,-53.090,36.2,-73.530,80.2,10.0F,0.0,5,1.7,255,3.1,EAR
2,3,2021,1,27,21,46,10.9,0.38,L,-54.689,5.2,-67.467,9.2,9.5,3.7,4,0.1,211,3.1,EAR
3,4,2021,2,8,23,10,31.6,1.00,L,-54.565,4.0,-66.912,8.4,8.4,5.8,7,0.4,180,3.8,EAR
4,5,2021,2,17,2,2,20.8,0.75,R,-54.636,4.9,-69.748,7.0,3.1,7.5,3,0.3,239,2.2,EAR


PASO 2: Explorar datos 

In [4]:
df.columns.tolist()  # Lista las columnas
len(df.columns)      # Cantidad de columnas

df.info() # Ver info general

df.isnull().sum() # Verificar datos nulos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   #         240 non-null    int64  
 1   Año       240 non-null    int64  
 2   M         240 non-null    int64  
 3   D         240 non-null    int64  
 4   H         240 non-null    int64  
 5   M.1       240 non-null    int64  
 6   S         240 non-null    float64
 7   Err.      240 non-null    float64
 8   L         240 non-null    object 
 9   Latitud   240 non-null    float64
 10  Err..1    240 non-null    object 
 11  Longitud  240 non-null    float64
 12  Err..2    240 non-null    float64
 13  Prof.     240 non-null    object 
 14  Err..3    240 non-null    float64
 15  NST       240 non-null    int64  
 16  RMS       240 non-null    float64
 17  GAP       240 non-null    int64  
 18  ML        232 non-null    float64
 19  Agen.     232 non-null    object 
dtypes: float64(8), int64(8), object(

#           0
Año         0
M           0
D           0
H           0
M.1         0
S           0
Err.        0
L           0
Latitud     0
Err..1      0
Longitud    0
Err..2      0
Prof.       0
Err..3      0
NST         0
RMS         0
GAP         0
ML          8
Agen.       8
dtype: int64

In [6]:
# Ver cuántas filas tenés de entrada
len(df)

# Por ejemplo, antes de eliminar filas con letras en 'L'
df['L'].value_counts(dropna=False)


L
R    124
L    115
D      1
Name: count, dtype: int64

In [8]:
# Verificar valores en las columnas que causan error
df['L'].unique()         # distancia_epicentral
df['Err..1'].unique()    # error_latitud
df['Prof.'].unique()     # profundidad_km


array(['0.0', '10.0F', '9.5', '8.4', '3.1', '9.0', '18.2', '12.0F', '6.0',
       '6.2', '8.1', '4.2', '6.1', '6.8', '5.7', '27.0', '18.9', '27.7',
       '12.2F', '11.5', '6.5', '7.9', '1.9', '32.1', '15.0', '5.0',
       '66.7', '10.3', '8.3', '14.5', '16.3', '12.1', '0.4', '12.3',
       '36.9F', '7.7', '4.0', '13.9', '6.9', '30.3', '25.2', '30.7',
       '9.1', '10.0', '10.4', '3.3', '11.9', '22.8', '1.2', '32.3', '8.5',
       '22.0', '14.2', '32.0', '2.9', '5.3', '2.2', '24.5', '7.8', '16.1',
       '15.8', '20.9', '13.0', '1.1', '12.4', '4.1', '10.9', '20.0',
       '20.6', '8.0', '13.6', '3.6', '8.2', '1.5', '115.0', '14.3', '1.6',
       '11.6', '15.7', '18.6', '5.2', '16.7', '29.2', '7.1', '0.7', '4.5',
       '20.7', '18.5', '11.2', '5.8', '24.4', '28.0', '11.0', '13.5',
       '28.9', '10.1', '20.8', '31.8', '16.2', '21.0', '7.0', '11.4F',
       '25.7', '28.6', '9.4', '23.1', '2.7', '32.8', '15.1', '19.5',
       '0.8', '57.0', '6.4', '3.0', '10.8', '12.9', '3.4', '4.8', '

In [10]:
# Renombrar columnas con nombres claros

df.columns = [
    'id', 'anio', 'mes', 'dia', 'hora', 'minuto', 'segundo',
    'error_segundo', 'clasificacion_distancia', 'latitud', 'error_latitud',
    'longitud', 'error_longitud', 'profundidad_km', 'error_profundidad',
    'cantidad_estaciones', 'rms', 'gap', 'magnitud_ml', 'agencia'
]

# Limpiar \"profundidad_km\" (quitar F y comas)
df['profundidad_km'] = df['profundidad_km'].astype(str)
df['profundidad_km'] = df['profundidad_km'].str.replace('F', '', regex=False)
df['profundidad_km'] = df['profundidad_km'].str.replace(',', '.')
df['profundidad_km'] = df['profundidad_km'].astype(float)

# Eliminar filas con ML faltante (NaN o vacías)
df = df[df['magnitud_ml'].notnull()]

# Crear variable objetivo significativo (ML ≥ 4.0)
df['significativo'] = (df['magnitud_ml'] >= 4.0).astype(int)

# Eliminar columnas innecesarias para el modelo
columnas_a_eliminar = [
    'id', 'error_segundo', 'error_latitud', 'error_longitud',
    'error_profundidad', 'agencia'
]
df = df.drop(columns=[col for col in columnas_a_eliminar if col in df.columns])

# Validar estructura
print(df.dtypes)
df.head()
df['significativo'].value_counts(normalize=True)


anio                         int64
mes                          int64
dia                          int64
hora                         int64
minuto                       int64
segundo                    float64
clasificacion_distancia     object
latitud                    float64
longitud                   float64
profundidad_km             float64
cantidad_estaciones          int64
rms                        float64
gap                          int64
magnitud_ml                float64
significativo                int32
dtype: object


significativo
0    0.965517
1    0.034483
Name: proportion, dtype: float64

In [12]:
df.to_csv("df_sismos_limpio.csv", index=False)
